<a href="https://colab.research.google.com/github/Macklees928/me/blob/main/GPT_2_HODL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HODL Final Project

## Import packages & load data

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
hodl_project_path = '/content/drive/My Drive/HODL Project/'
rawData = pd.read_csv(hodl_project_path + "/full_dataset.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rawData.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
!pip uninstall transformers accelerate -y
!pip install transformers[torch] accelerate -U


Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.1 MB/s eta 0:00:00


## Modeling

GPT2

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

from transformers import Trainer, TrainingArguments

from tqdm.auto import tqdm

import pandas as pd
import numpy as np
import pandas as pd
import os
from google.colab import drive

Pre-Trained GPT 2 Model

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

prompt_text = "Create a recipe with these ingredients: chicken, spinach, peppers, and onions"
input_ids = tokenizer.encode(prompt_text, return_tensors="pt")

attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

output = model.generate(input_ids, max_length=100, num_return_sequences=1, attention_mask=attention_mask)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated text:")
print(generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text:
Create a recipe with these ingredients: chicken, spinach, peppers, and onions.

Step 1: Prepare the chicken.

Step 2: Add the chicken to the pot.

Step 3: Add the chicken to the pot.

Step 4: Add the chicken to the pot.

Step 5: Add the chicken to the pot.

Step 6: Add the chicken to the pot.

Step 7: Add the chicken to the pot.

Step


Training GPT 2 on Recipe Data

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding(50260, 768)

In [ ]:
model_save_path = './leesGPT'

In [ ]:
tokenizer.save_pretrained(model_save_path)

('./leesGPT/tokenizer_config.json',
 './leesGPT/special_tokens_map.json',
 './leesGPT/vocab.json',
 './leesGPT/merges.txt',
 './leesGPT/added_tokens.json',
 './leesGPT/tokenizer.json')

In [ ]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [ ]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [ ]:
clean = rawData
clean = clean.sample(frac=.001)
clean.reset_index(drop=True,inplace=True)

In [ ]:
def print_recipe(idx):
    print(f"{clean['NER'][idx]}\n\n{clean['directions'][idx]}")

def form_string(ingredient,direction):
    s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nDirections:\n{direction.strip()}<|endoftext|>"
    return s

In [ ]:
data = clean.apply(lambda x:form_string(x['NER'],x['directions']),axis=1).to_list()

In [ ]:
train_size = 0.80
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [ ]:
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [ ]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/1784 [00:00<?, ?it/s]

  0%|          | 0/447 [00:00<?, ?it/s]

In [ ]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=3,
                         save_strategy='no'
                        )

In [ ]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [ ]:
trainer.train()

Step,Training Loss
500,0.530500


TrainOutput(global_step=669, training_loss=0.4729214769485998, metrics={'train_runtime': 1817.0103, 'train_samples_per_second': 2.945, 'train_steps_per_second': 0.368, 'total_flos': 2796869910528000.0, 'train_loss': 0.4729214769485998, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

In [ ]:
pl = pipeline(task='text-generation',model='./leesGPT')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def create_prompt(ingredients):
    first_sentence = "Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”."
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    #s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    s = f"<|startoftext|>{first_sentence}:\n{ingredients}\n"
    return s

ingredients = ['Chicken, garlic, spinach, peppers, olive oil, salt, pepper']

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.:
chicken
garlic
spinach
peppers
olive oil
salt
pepper
butter", "sour cream", "sherry", "freshly ground black pepper", "sour cream"]

Directions:
["Preheat oven to 350\u00b0F. Line a baking sheet with foil and place the chicken in the baking sheet with foil to keep the chicken from sticking to the foil.", "Place the chicken in the baking sheet and set the grill on the grill on the top of the chicken until the chicken is browned and the juices run clear.", "While the chicken is cooking, prepare the rest of the ingredients. Add the sour cream and the onion, cover the pan and cook the chicken until soft and golden brown, about 10 minutes per side.", "Remove chicken from the oven and set the grill on the top of the chicken until the chicken is done.", "Serve hot.", "Yield: 4 to 6 servings."]


In [ ]:
def create_prompt(ingredients):
    first_sentence = "Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”."
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    #s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    s = f"<|startoftext|>{first_sentence}:\n{ingredients}\n"
    return s

ingredients = ['Flour, sugar, butter, eggs, milk, chocolate chips, oil, whipped cream, frosting']

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.:
flour
sugar
butter
eggs
milk
chocolate chips
oil
whipped cream
frosting
salt", "vanilla flavoring", "butter", "flour", "soda", "sugar", "vanilla"]

Directions:
["Combine first 4 ingredients.", "Add milk, vanilla and oil.", "Add cream and mix thoroughly.", "Pour into a 9 x 13 x 2-cup casserole dish.", "Bake in a 350\u00b0 oven for 35 minutes or until toothpick inserted in center comes out clean.", "Cool on a wire rack."]


In [ ]:
def create_prompt(ingredients):
    first_sentence = "Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”."
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    #s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    s = f"<|startoftext|>{first_sentence}:\n{ingredients}\n"
    return s

ingredients = ['Broccoli, strawberries, pineapple, JELLO mix, raisins, pork, cod']

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.:
broccoli
strawberries
pineapple
jello mix
raisins
pork
cod
sour cream (or possibly corn syrup)
salt", "pork tenderloin"]

Directions:
["Cook broccoli in boiling water until soft and fluffy.", "Drain.", "Add jello and toss to mix.", "Drain, add reserved 1 cup of jello mixture and mix well. Add 1 cup of sour cream and mix well.", "Pour in a small casserole dish and top with 1/4 cup of sour cream and remaining sour cream. Top with remaining sour cream. Pour over broccoli and bake for 1/2 hour or until bubbly.", "Serves 2 to 4 people."]


In [ ]:
def create_prompt(ingredients):
    first_sentence = "Make me a recipe with these ingredients:"
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    #s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    s = f"<|startoftext|>{first_sentence}:\n{ingredients}\n"
    return s

ingredients = ['Chicken, garlic, spinach, peppers, olive oil, salt, pepper']

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Make me a recipe with these ingredients::
chicken
garlic
spinach
peppers
olive oil
salt
pepper

fresh cilantro", "fresh cilantro leaves", "fresh parsley"]

Directions:
["In a large skillet, cook the garlic in olive oil, stirring occasionally, until it begins to brown.", "Stir in the onion, and cook over medium heat until the onion is soft, about 5 minutes.", "Add peppers and cook, stirring constantly, about 5 minutes.", "Remove the peppers and cook, stirring constantly, until the peppers are tender.", "Add the cilantro, parsley, and cilantro, and toss to coat."]


In [ ]:
def create_prompt(ingredients):
    first_sentence = ("You are an expert chef who is highly skilled at providing cooking instructions. When given a list of ingredients, please: "
                      "Provide a step by step list of instructions for cooking a meal. "
                      "The instructions will only require the listed ingredients. "
                      "If you can’t think of a possible recipe, say: you need to go shopping ASAP.")

    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',', '\n')
    s = first_sentence + ":\n" + ingredients + "\n"
    return s


ingredients = ['Chicken, garlic, spinach, peppers, olive oil, salt, pepper']

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

You are an expert chef who is highly skilled at providing cooking instructions. When given a list of ingredients, please: Provide a step by step list of instructions for cooking a meal. The instructions will only require the listed ingredients. If you can’t think of a possible recipe, say: you need to go shopping ASAP.:
chicken
garlic
spinach
peppers
olive oil
salt
pepper
fresh ground black pepper", "tomatoes"]

Directions:
["Heat the oil in a large nonstick frying pan and fry the spinach until it is browned. Add the garlic and fry for another 3 minutes or until the garlic starts to soften. Add the tomatoes to the oil, and fry for another 3 minutes or until the tomatoes are tender. Add the spinach, and fry for another 5 minutes or until tender.", "Drain the liquid and add the spinach to the pan, and cook for another 15 minutes or until the water has absorbed all the liquid.", "Drain the spinach and add the tomatoes and the olive oil to the pan.", "Add some more olive oil if needed.", "